In [1]:
import pandas as pd
import sqlite3 as sql
import sqlalchemy as alc
import plotly.figure_factory as fig
import pyarrow.feather as feather
import utils.feather_cache as feather_cache
import seaborn as sns
import numpy as np
import display.heatmap as heatmap
import matplotlib.pyplot as plt
from data_types import tables
from data_types import pcap_sqlite_db
from display import formatter
from views.algo_order_fills import display as algo_order_display
from matplotlib.colors import ListedColormap, LinearSegmentedColormap


In [3]:
sim_con = sql.connect('data/310/2020-09-16_310_24h.sqlite')
pcap_tables = pcap_sqlite_db.read_tables(con=sim_con)


TypeError: Argument 'table' has incorrect type (expected pyarrow.lib.Table, got str)

In [6]:
for kv in pcap_tables.items():
    if kv[0] == 'event_items':
        tables.cast_column_types(kv[0], kv[1])
    else:
        tables.cast_column_types(None, kv[1])

In [16]:
for kv in pcap_tables.items():
    feather.write_feather(kv[1], f'data/cache/310/09-16/{kv[0]}.feather')

In [2]:
def sample_limit(df: pd.DataFrame, max_rows = 25):
    if len(df) > max_rows:
        return df.sample(max_rows)
    return df

def print_pretty(df: pd.DataFrame, max_rows = 25, print_row_count = False):
    col_styles = {}
    for c in df.columns:
       if 'price' in c or 'avg' in c or 'profit' in c or 'bid_p' in c or 'ask_p' in c:
          col_styles[c] = "{:.2f}"
       elif 'timestamp' in c:
          col_styles[c] = lambda t: f'{t.total_seconds():.2f}'

    for c in df.columns:
       if 'price' in c or 'avg' in c or 'profit' in c or 'var' in c:
          col_styles[c] = "{:.2f}"
       elif 'timestamp' in c:
          col_styles[c] = lambda t: f'{t.total_seconds():.2f}'
    if print_row_count:
        print(f'Total Rows: {len(df)} (sample {max_rows}).')
    df = sample_limit(df)
    colset = list(set(['fill_profit_delta'] + tables.bbo_var_cols(1) + tables.bbo_var_inter_period_delta(1, 5)).intersection(set([c.split(' ')[0] for c in df.columns])))
    print(f"set is {colset}")
    return df.style.format(col_styles).set_table_styles([
       dict(selector='tr', props=[('text-align', 'right')])
       #dict(selector='tr', props=[('background', '#88ff0000')])
    ]).background_gradient(cmap=LinearSegmentedColormap.from_list('', [(0,0,1), (1,1,1), (1,0,0)]), subset=colset)
    #.background_gradient(cmap=LinearSegmentedColormap.from_list('', [(0,0,1), (1,1,1), (1, 0, 0)]), subset=['bid_p_0_var_1s'])

In [17]:
for table in ['events', 'event_items']:
    pcap_tables[table]['t_timestamp'] = pcap_tables[table]['t_time'].apply(lambda t: pd.Timestamp(t, unit='ns'))

In [19]:
book_stats = feather.read_feather('data/cache/310/09-16/book_stats.feather')
sec_book_stat_groups = book_stats.groupby('sid')
sec_book_stats = {
    'u0': sec_book_stat_groups.get_group(12181)
}
sec_book_stats['u0'].set_index('eid', inplace=True)
sec_book_stats['u0'].sort_values(by='eid', inplace=True)
sec_book_stats['u0'] = tables.merge_book_stat_sides(sec_book_stats['u0'])
for kv in sec_book_stats.items():
    tables.add_row_book_stat_deltas(kv[1])
    kv[1] = kv[1].merge(pcap_tables['events'][['eid', 't_timestamp']], on='eid')

NameError: name 'events' is not defined